In [1]:
#Imports
import pandas as pd
from datetime import datetime
import csv

#Sqlalchemy imports
from sqlalchemy import Column, Float, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect,func
# Flask imports
from flask import Flask, jsonify, render_template

In [2]:
#Create engine
    #Set echo=True for debugging
engine = create_engine("sqlite:///../sqldata/restaurants.db",echo=True)
#Link inspector to engine
inspector = inspect(engine)

#Not used# Declare a Base using `automap_base()`
#Not used# Base = automap_base()
#Declarative base ---> columns lack types
Base = declarative_base()

#Override class to establish primary for reflection
class Vegetarian(Base):
    __tablename__ = "vegetarian"
   # __table_args__ = {'extend_existing': True}
    restaurant_name = Column("restaurant_name",String, primary_key=True)
    address = Column(String)
    city = Column(String)
    state = Column(String)
    zip_code = Column(Integer)
    phone = Column(Integer)
    cuisine_type = Column(String)
    rating = Column(String)
    price = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
#Not used#Continue automapping:
#Not used#Base.prepare(engine, reflect=True)

# Use the Base class to reflect the database tables
#Base.metadata.create_all(engine)


2018-03-24 00:38:46,240 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-24 00:38:46,248 INFO sqlalchemy.engine.base.Engine ()
2018-03-24 00:38:46,251 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-24 00:38:46,254 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#Verify table creation with inspector
inspector.get_table_names()

2018-03-24 00:38:46,271 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-03-24 00:38:46,273 INFO sqlalchemy.engine.base.Engine ()


['vegetarian']

In [4]:
columns = inspector.get_columns('vegetarian')

2018-03-24 00:38:46,293 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 00:38:46,298 INFO sqlalchemy.engine.base.Engine ()


In [5]:
print("========================")
print("Column names:")
for c in columns:
    print(c['name'])
print("========================")
print (columns)


Column names:
restaurant_name
address
city
state
zip_code
phone
cuisine_type
rating
price
latitude
longitude
[{'name': 'restaurant_name', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'address', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'city', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'state', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'zip_code', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'phone', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'cuisine_type', 'type': NullType(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'rating', 'type': NullType(), 'nullable':

In [6]:
Base.classes.keys()

AttributeError: type object 'Base' has no attribute 'classes'

In [7]:
Base.metadata.create_all(engine)
#Create session (to pusb objects and query server)
session = Session(bind=engine)
conn = engine.connect()

2018-03-24 00:39:04,777 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 00:39:04,778 INFO sqlalchemy.engine.base.Engine ()


In [8]:
#csv ---> dataframe ---> to_dict(orient='records') --> 
def populate(engine, table, csvfile):
    conn = engine.connect()
    df =pd.read_csv(csvfile,encoding='latin-1')
    data = df.to_dict(orient='records')
    conn.execute(table.insert(), data)

In [9]:
#run populate function on db
populate(engine,Vegetarian.__table__,"../CleanData/clean_no_index.csv")

2018-03-24 00:39:06,231 INFO sqlalchemy.engine.base.Engine INSERT INTO vegetarian (restaurant_name, address, city, state, zip_code, phone, cuisine_type, rating, price, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-03-24 00:39:06,231 INFO sqlalchemy.engine.base.Engine (('Blue Collar', '6730 Biscayne Blvd', 'Miami', 'FL', '33138', 13057560366.0, 'American (Traditional)', 4.0, '$$', 26.07967414, -80.25222212), ('Atlantic No 5', '605 W Main St', 'Louisville', 'KY', '40202', 15028833398.0, 'Breakfast & Brunch', 4.0, '$$', 25.750963, -80.258375), ("Famous Dave's", '17770 Southcenter Pkwy', 'Tukwila', 'WA', '98188', 12066312000.0, 'Barbeque', 3.0, '$$', 34.02829213, -81.09327574), ('Quiznos', '1595 N Peach Ave', 'Fresno', 'CA', '93727', 15594548477.0, 'Fast Food', 3.0, '$', 44.939642, -93.13603), ('Hobbit Cafe', '2243 Richmond Ave', 'Houston', 'TX', '77098', 17135265460.0, 'Breakfast & Brunch', 3.5, '$$', 40.11574901, -83.08946112), ('Pizzeria 3301', '3301 W Canal St', 'M

In [10]:
#verify table information:
data = engine.execute("PRAGMA table_info([vegetarian]);")
for item in data:
    print(item)

2018-03-24 00:39:07,856 INFO sqlalchemy.engine.base.Engine PRAGMA table_info([vegetarian]);
2018-03-24 00:39:07,859 INFO sqlalchemy.engine.base.Engine ()
(0, 'restaurant_name', '', 0, None, 0)
(1, 'address', '', 0, None, 0)
(2, 'city', '', 0, None, 0)
(3, 'state', '', 0, None, 0)
(4, 'zip_code', '', 0, None, 0)
(5, 'phone', '', 0, None, 0)
(6, 'cuisine_type', '', 0, None, 0)
(7, 'rating', '', 0, None, 0)
(8, 'price', '', 0, None, 0)
(9, 'latitude', '', 0, None, 0)
(10, 'longitude', '', 0, None, 0)


In [11]:
Base.metadata.create_all(engine)
session.commit()

2018-03-24 00:39:08,236 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("vegetarian")
2018-03-24 00:39:08,237 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
#first_row = session.query(CLASSNAMEHERE).first()
## Display the row's columns and data in dictionary format
#first_row.__dict__